# 4-4: Sparse Categorical Cross Entropy

### Code.4-4-1: SCCE Calcrulation

In [40]:
import tensorflow as tf

from tensorflow.keras.losses import SparseCategoricalCrossentropy

batch_size, n_class = 16,5

predictions = tf.random.uniform(shape=(batch_size,n_class), #마지막 layer는 각데이터가 class개수만큼 나와야하기때문에, (batch_size,n_class)으로 shape가 형성되어야한다.
                               minval=0,maxval=1,
                               dtype=tf.float32)
pred_sum = tf.reshape(tf.reduce_sum(predictions,axis=1),(-1,1)) #한 layers내에 있는 각 neurons 결괏값들의 합.
#print(predictions.shape,pred_sum.shape) 결과를 보면, 5개의 n_class가 pred_sum을 통해 1개의 class로 변했음을 알 수 있다.
predictions = predictions/pred_sum #항상 한 layer내에 있는 각 neurons 결괏값들의 합은 1이돼야하기 때문에, pred_sum으로 나워준다. 이렇게 되면, predictions의 합은 무조건 1이 되기때문 ㅎ

labels = tf.random.uniform(shape=(batch_size,),
                          minval=0,maxval=n_class,
                          dtype=tf.int32)   #labels는 0부터 4까지 정수로 구성된 (batch_size,)의 형태로 만들어준다.

#tenserflow내에 있는 패키지 활용.
loss_object = SparseCategoricalCrossentropy()
loss = loss_object(labels, predictions)
print("SCCE(Tensorflow):",loss.numpy())

#실제 공식을 활용하여 loss값구하기.
ce = 0
for label, prediction in zip(labels,predictions):
    ce += -tf.math.log(prediction[label])
ce /=batch_size
print("SCCE(Menual):",ce.numpy())

SCCE(Tensorflow): 1.8018494
SCCE(Menual): 1.8018494


### Code.4-4-2: SCCE with Model/Dataset

In [48]:
import tensorflow as tf

from tensorflow.keras.layers import Dense

from tensorflow.keras.losses import SparseCategoricalCrossentropy

N,n_feature = 100,2
n_class = 5

X = tf.zeros(shape=(0,n_feature))
Y = tf.zeros(shape = (0,1),dtype=tf.int32)

tf.random.set_seed(2)
for class_idx in range(n_class):
    center = tf.random.uniform(minval=-15,maxval=15,shape=(2,))
    
    x1 = center[0] + tf.random.normal(shape=(N,1))
    x2 = center[1] + tf.random.normal(shape=(N,1))
    
    x = tf.concat((x1,x2),axis=1)
    y = (class_idx+1) * tf.ones(shape=(N,1),dtype=tf.int32)
    
    X = tf.concat((X,x),axis=0)
    Y = tf.concat((Y,y),axis=0)
#여기까지는 dataset을 구성한 것.

batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices((X,Y))
dataset = dataset.batch(batch_size)

model = Dense(units = n_class, activation = "softmax") #multi-class이기 때문에, softmax를 activation으로 사용한다. neurons의 개수는 n_class.
loss_object = SparseCategoricalCrossentropy()

loss_sum = 0
loss_row = 0
for x, y in dataset:
    predictions = model(x)
    loss = loss_object(y,predictions)
    print(loss.numpy())
    loss_row += 1
    loss_sum += loss
print("SCCE:",loss_sum/batch_size)
print("loss_row:",loss_row)

9.81967
10.124693
10.492644
10.605381
10.190081
9.7218685
3.0947025
0.41211507
0.44514546
0.3981114
0.4485567
0.43118683
1.579084
3.4739544
3.4371014
2.8629012
3.7699566
2.8660102
4.644404
8.471378
8.574875
8.4469
8.283646
8.400702
8.165389
0.0
0.0
0.0
0.0
0.0
0.0
0.0
SCCE: tf.Tensor(8.697529, shape=(), dtype=float32)
loss_row: 32


# 4-5: Categorical Cross Entropy
one-hot encoding 된 상태에서 loss값을 계산해보자.

### Code 4-5-1: CCE Calculation

In [63]:
import tensorflow as tf

from tensorflow.keras.losses import CategoricalCrossentropy

batch_size, n_class = 16,5

predictions = tf.random.uniform(shape=(batch_size,n_class),
                               minval=0,maxval=1,
                               dtype=tf.float32)
pred_sum = tf.reshape(tf.reduce_sum(predictions,axis=1),(-1,1))
predictions = predictions/pred_sum

labels = tf.random.uniform(shape=(batch_size,),
                          minval=0,maxval=n_class,
                          dtype=tf.int32)

labels = tf.one_hot(labels, n_class) #one-hot vector로 변형

loss_object = CategoricalCrossentropy() #CategoricalCrossentropy는 Y label이 one-hot encoding이 되어 있을때 하는것.
loss = loss_object(labels, predictions)

print("CCE(Tensorflow):",loss.numpy())

cce_man = tf.reduce_mean(tf.reduce_sum(-labels *tf.math.log(predictions),axis=1))

print("CCE(Menual):",cce_man.numpy())

CCE(Tensorflow): 1.7213991
CCE(Menual): 1.7213991


### Code.4-5-2: CCE with Model/Dataset

In [66]:
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy

N,n_feature = 8,2
n_class = 5

X = tf.zeros(shape=(0,n_feature))
Y = tf.zeros(shape = (0,),dtype=tf.int32)

tf.random.set_seed(1) 

for class_idx in range(n_class):
    center = tf.random.uniform(minval=-15,maxval=15,shape=(2,))
 
    x1 = center[0] + tf.random.normal(shape=(N,1))
    x2 = center[1] + tf.random.normal(shape=(N,1)) 
    
    x = tf.concat((x1,x2),axis=1)
    y = class_idx * tf.ones(shape=(N,),dtype=tf.int32)
    
    X = tf.concat((X,x),axis=0)
    Y = tf.concat((Y,y),axis=0)
    
Y = tf.one_hot(Y,depth=n_class, dtype=tf.int32) #one-hot encoding을 실행한 것.

dataset = tf.data.Dataset.from_tensor_slices((X,Y))
dataset = dataset.batch(batch_size)

model = Dense(units = n_class, activation = "softmax") #multi-class이기 때문에, softmax를 activation으로 사용한다. neurons의 개수는 n_class.
loss_object = CategoricalCrossentropy()

for x, y in dataset:
    predictions = model(x)
    loss = loss_object(y,predictions)
    print(loss.numpy())

3.31924
9.251816
17.666653
